In [ ]:
from keras.datasets import mnist
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
from keras.models import Sequential,Model
from keras.layers import Dropout,Dense,Flatten,BatchNormalization,Activation,Input
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.utils.vis_utils import plot_model
from keras.utils import np_utils
from matplotlib import pyplot
K.set_image_dim_ordering('tf')

(X_train,y_train),(X_test,y_test) = mnist.load_data()

X_train = X_train.reshape(X_train.shape[0],28,28,1).astype('float32')
X_test = X_test.reshape(X_test.shape[0],28,28,1).astype('float32')

X_train = X_train/255
X_test = X_test/255

y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

num_classes = y_test.shape[1]
print("Number of classes " ,num_classes)

def cnn_model():
    model = Sequential()
    
    model.add(Conv2D(32,(7,7), input_shape=(28,28,1), kernel_initializer='he_uniform'))
    model.add(Activation('tanh'))
    model.add(BatchNormalization())
    
    model.add(MaxPooling2D(pool_size=(2,2),padding='same'))
    
    model.add(Conv2D(32,(5,5), kernel_initializer='he_uniform'))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    
    model.add(MaxPooling2D(pool_size=(2,2),padding='same'))
    
    model.add(Conv2D(32,(3,3), kernel_initializer='he_uniform'))
    model.add(Activation('tanh'))
    model.add(BatchNormalization())
    
    model.add(MaxPooling2D(pool_size=(2,2),padding='same'))
    
    model.add(Flatten())
    
    model.add(Dense(128, activation='relu',kernel_initializer='glorot_uniform'))
    model.add(Dropout(0.2))
    model.add(Dense(10,activation='relu',kernel_initializer='he_normal'))
    model.add(Dropout(0.1))
    model.add(Dense(num_classes,activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
    return model

def ann_model():
    Images = Input(shape=(28,28,1))
    flat_n = Flatten()(Images)
    dense1 = Dense(100,activation='relu',kernel_initializer='glorot_uniform')(flat_n)
    dense2 = Dense(2000, activation='relu',kernel_initializer='he_normal')(dense1)
    dense3 = Dense(100,activation='relu')(dense2)
    dense4 = Dense(10,activation='relu')(dense3)
    dropout = Dropout(0.3)(dense4)
    dense5 = Dense(num_classes,activation='softmax')(dropout)
    model = Model(inputs=Images,outputs=dense5)
    model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])
    return model

cnn_model = cnn_model()
ann_model = ann_model()

plot_model(cnn_model, to_file='cnn_model_plot.png', show_shapes=True, show_layer_names=True)
plot_model(ann_model, to_file='ann_model_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
datagen = ImageDataGenerator(featurewise_center=True,
                            featurewise_std_normalization=True,
                            #zca_whitening=True,
                            rotation_range=30,
                            width_shift_range=0.1,
                            height_shift_range=0.1,
                            horizontal_flip=True, 
                            vertical_flip=False
                            )
train_gen = datagen.flow(X_train,y_train,batch_size=32)
cnn_model.fit_generator(train_gen,
                        steps_per_epoch=60000,
                        epochs=500,
                        validation_data = (X_test,y_test))
cnn_model.save_weights("cnn_model.h5")
ann_model.fit_generator(train_gen,
                        steps_per_epoch=60000,
                        epochs=500,
                        validation_data = (X_test,y_test))
ann_model.save_weights("ann_model.h5")